In [ ]:
import os
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import rasterio
from tqdm import tqdm
import pickle

In [ ]:
# Helper functions
def read(file):
    with rasterio.open(file) as src:
        return src.read(), src.profile

def get_dates(path, n=None):
    files = os.listdir(path)
    dates = list()
    for f in files:
        f = f.split("_")[0]
        if len(f) == 8:  # YYYYMMDD format
            dates.append(f)
    dates = set(dates)
    if n is not None:
        dates = random.sample(dates, n)
    dates = list(dates)
    dates.sort()
    return dates

class SentinelDataset(Dataset):
    def __init__(self, root_dirs, seqlength=30, tileids_paths=None, preprocess_dirs=None, save_preprocessed=False):
        self.root_dirs = [r.rstrip("/") for r in root_dirs]
        self.data_dirs = ['data2016']
        self.seqlength = seqlength
        self.samples = list()
        self.ndates = list()
        self.classids, self.classes = self.read_classes()
        self.preprocess_dirs = preprocess_dirs
        self.save_preprocessed = save_preprocessed
        self.tileids_paths = tileids_paths
        self.prepare_dataset()

    def read_classes(self):
        classes_path = 'data/sentinel2/lombardia-classes/classes25pc.txt'
        with open(classes_path, 'r') as f:
            classes = f.readlines()
        ids, names = [], []
        for row in classes:
            row = row.replace("\n", "")
            if '|' in row:
                cls_info = row.split('|')
                id_info = [int(x) for x in cls_info[0].split(',')]
                ids.append(id_info)
                names.append(cls_info[1])
        return ids, names

    def prepare_dataset(self):
        for root_dir, tileids_path, preprocess_dir in zip(self.root_dirs, self.tileids_paths, self.preprocess_dirs):
            print("Reading dataset info:", root_dir)
            dirs = []
            if tileids_path is None:
                for d in self.data_dirs:
                    dirs_name = os.listdir(os.path.join(root_dir, d))
                    dirs_path = [os.path.join(root_dir, d, f) for f in dirs_name]
                    dirs.extend(dirs_path)
            else:
                try:
                    with open(tileids_path, 'r') as f:
                        files = [el.replace("\n", "") for el in f.readlines()]
                    for d in self.data_dirs:
                        dirs_path = [os.path.join(root_dir, d, f) for f in files]
                        dirs.extend(dirs_path)
                except PermissionError as e:
                    print(f"Permission error: {e}")
                    continue
                except FileNotFoundError as e:
                    print(f"File not found: {e}")
                    continue
            stats = dict(rejected_nopath=0, rejected_length=0, total_samples=0)
            for path in tqdm(dirs):
                if not os.path.exists(path) or not os.path.exists(os.path.join(path, "y.tif")):
                    stats["rejected_nopath"] += 1
                    continue
                ndates = len(get_dates(path))
                if ndates < self.seqlength:
                    stats["rejected_length"] += 1
                    continue
                stats["total_samples"] += 1
                self.samples.append((path, preprocess_dir))
                self.ndates.append(ndates)
            self.print_stats(stats)
            if self.save_preprocessed:
                self.save_preprocessed_data()

    def save_preprocessed_data(self):
        for sample, preprocess_dir in self.samples:
            label, profile = read(os.path.join(sample, "y.tif"))
            dates = get_dates(sample, n=self.seqlength)
            data = {
                'label': label,
                'profile': profile,
                'dates': dates
            }
            save_path = os.path.join(preprocess_dir, os.path.basename(sample) + ".pkl")
            os.makedirs(preprocess_dir, exist_ok=True)
            try:
                with open(save_path, 'wb') as f:
                    pickle.dump(data, f)
                print(f"Saved preprocessed data to {save_path}")
            except PermissionError as e:
                print(f"Permission error: {e}")
            except FileNotFoundError as e:
                print(f"File not found: {e}")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample_path, preprocess_dir = self.samples[idx]
        label, profile = read(os.path.join(sample_path, "y.tif"))
        dates = get_dates(sample_path, n=self.seqlength)
        data_10m, data_20m, data_60m = [], [], []
        for date in dates:
            data_10m.append(read(os.path.join(sample_path, date + "_10m.tif"))[0])
            data_20m.append(read(os.path.join(sample_path, date + "_20m.tif"))[0])
            data_60m.append(read(os.path.join(sample_path, date + "_60m.tif"))[0])
        data_10m = np.array(data_10m) * 1e-4
        data_20m = np.array(data_20m) * 1e-4
        data_60m = np.array(data_60m) * 1e-4
        data_20m = torch.from_numpy(data_20m)
        data_60m = torch.from_numpy(data_60m)
        data_20m = torch.nn.functional.interpolate(data_20m, size=data_10m.shape[2:4])
        data_60m = torch.nn.functional.interpolate(data_60m, size=data_10m.shape[2:4])
        data = torch.cat((torch.from_numpy(data_10m), data_20m, data_60m), 1)
        data = data.permute(1, 0, 2, 3).float()
        label = torch.from_numpy(label).long()
        return data, label

    def print_stats(self, stats):
        print_lst = list()
        for k, v in zip(stats.keys(), stats.values()):
            print_lst.append("{}:{}".format(k, v))
        print('\n', ", ".join(print_lst))

In [ ]:
# Define paths and parameters
root_paths = ['data/sentinel2/lombardia', 'data/sentinel2/lombardia2']
preprocess_dirs = ['data/sentinel2/lombardia_preprocessed', 'data/sentinel2/lombardia2_preprocessed']
tileids_paths = ['data/sentinel2/lombardia/tileids/train_fold0.tileids', 'data/sentinel2/lombardia2/tileids/train_fold0.tileids']

# Initialize and prepare dataset
dataset = SentinelDataset(root_paths, seqlength=30, tileids_paths=tileids_paths, preprocess_dirs=preprocess_dirs, save_preprocessed=True)

# Create DataLoader for training and validation
batch_size = 16
train_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ASPP(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(ASPP, self).__init__()
        self.atrous_block1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, dilation=1, bias=False)
        self.atrous_block6 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=6, dilation=6, bias=False)
        self.atrous_block12 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=12, dilation=12, bias=False)
        self.atrous_block18 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=18, dilation=18, bias=False)
        self.conv_1x1_output = nn.Conv2d(out_channels * 4, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.batch_norm = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU()

    def forward(self, x):
        x1 = self.atrous_block1(x)
        x2 = self.atrous_block6(x)
        x3 = self.atrous_block12(x)
        x4 = self.atrous_block18(x)
        x = torch.cat((x1, x2, x3, x4), dim=1)
        x = self.conv_1x1_output(x)
        x = self.batch_norm(x)
        return self.relu(x)


In [ ]:
from torchvision.models import resnet50

class DeepLabV3(nn.Module):
    def __init__(self, num_classes, in_channels=3, pretrained=True):
        super(DeepLabV3, self).__init__()
        self.backbone = resnet50(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(self.backbone.children())[:-2])
        self.aspp = ASPP(2048, 256)
        self.conv1 = nn.Conv2d(256, 256, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(256)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(256, num_classes, kernel_size=1)

    def forward(self, x):
        x = self.backbone(x)
        x = self.aspp(x)
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.conv2(x)
        return F.interpolate(x, scale_factor=8, mode='bilinear', align_corners=True)


In [ ]:
num_classes = 22
model = DeepLabV3(num_classes=num_classes, in_channels=3, pretrained=True)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
import optuna
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import random_split

# Assuming 'dataset' is already created from Step 1

# Split dataset into training and validation sets
val_split = 0.2
train_size = int((1 - val_split) * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

def objective(trial):
    # Hyperparameters to be tuned
    lr = trial.suggest_loguniform('lr', 1e-5, 1e-2)
    weight_decay = trial.suggest_loguniform('weight_decay', 1e-5, 1e-2)
    gamma = trial.suggest_uniform('gamma', 0.1, 0.9)
    step_size = trial.suggest_int('step_size', 5, 20)
    batch_size = trial.suggest_categorical('batch_size', [8, 16, 32])

    # Create DataLoader with suggested batch size
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

    # Initialize model, criterion, optimizer, and scheduler
    model = DeepLabV3(num_classes=num_classes, in_channels=3, pretrained=True)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

    # Move model to device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    # Training loop for one epoch
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation loop for one epoch
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / total

    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")
    return val_loss


In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=5)

print("Best parameters:", study.best_params)
print("Best validation loss:", study.best_value)

In [ ]:
# Best hyperparameters from Optuna
best_params = study.best_params

# Create DataLoader with the best batch size
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'], shuffle=False, num_workers=4)

# Initialize the model, criterion, optimizer, and scheduler with best hyperparameters
model = DeepLabV3(num_classes=num_classes, in_channels=3, pretrained=True)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])
scheduler = StepLR(optimizer, step_size=best_params['step_size'], gamma=best_params['gamma'])

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Training loop
num_epochs = 25
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / total

    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    
    # Adjust the learning rate
    scheduler.step()

print('Training complete')


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
import optuna
from sklearn.model_selection import train_test_split
from tqdm import tqdm


In [ ]:
# Split the dataset into training and validation sets
train_indices, val_indices = train_test_split(list(range(len(dataset))), test_size=0.2, random_state=42)
train_dataset = torch.utils.data.Subset(dataset, train_indices)
val_dataset = torch.utils.data.Subset(dataset, val_indices)

# Create DataLoader with the best batch size
train_loader = DataLoader(train_dataset, batch_size=best_params['batch_size'], shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=best_params['batch_size'], shuffle=False, num_workers=4)


In [ ]:
# Initialize the model, criterion, optimizer, and scheduler with best hyperparameters
model = DeepLabV3(num_classes=num_classes, in_channels=3, pretrained=False)
model.load_state_dict(torch.load('/path/to/resnet50-0676ba61.pth'))  # Load pretrained weights

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=best_params['lr'], weight_decay=best_params['weight_decay'])
scheduler = StepLR(optimizer, step_size=best_params['step_size'], gamma=best_params['gamma'])

# Move model to the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


In [ ]:
# Training loop
num_epochs = 500
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}', leave=False):
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)

    # Validation
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_accuracy = correct / total

    print(f'Epoch {epoch + 1}/{num_epochs}, Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}')
    
    # Adjust the learning rate
    scheduler.step()

print('Training complete')


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def evaluate_model(model, dataloader, device):
    model.eval()
    all_labels = []
    all_preds = []
    
    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(preds.cpu().numpy())
    
    return np.array(all_labels), np.array(all_preds)

# Compute metrics
def compute_metrics(labels, preds):
    accuracy = accuracy_score(labels, preds)
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average='weighted')
    
    cm = confusion_matrix(labels, preds)
    
    return accuracy, precision, recall, f1, cm

# Plot confusion matrix
def plot_confusion_matrix(cm, class_names):
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()


In [ ]:
# Evaluate the model
val_labels, val_preds = evaluate_model(model, val_loader, device)

# Compute metrics
accuracy, precision, recall, f1, cm = compute_metrics(val_labels, val_preds)

# Print metrics
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Plot confusion matrix
class_names = dataset.classes
plot_confusion_matrix(cm, class_names)
